In [1]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [4]:
train_dir = '/content/drive/MyDrive/strawberry/sl_train'
validation_dir = '/content/drive/MyDrive/strawberry/sl_test'

In [5]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)


In [7]:
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=IMG_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              target_size=IMG_SIZE,
                                                              batch_size=BATCH_SIZE,
                                                              class_mode='categorical')

Found 338 images belonging to 2 classes.
Found 252 images belonging to 2 classes.


In [8]:
# Load the ResNet50 model and freeze its layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

# Add a custom classifier on top of the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(2, activation='softmax')
])


In [9]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=validation_generator)

Epoch 1/10
11/11 [==============================] - 289s 27s/step - loss: 0.6957 - accuracy: 0.5710 - val_loss: 0.5279 - val_accuracy: 0.7817
Epoch 2/10
11/11 [==============================] - 282s 27s/step - loss: 0.5581 - accuracy: 0.6716 - val_loss: 0.4538 - val_accuracy: 0.7738
Epoch 3/10
11/11 [==============================] - 280s 27s/step - loss: 0.4917 - accuracy: 0.7722 - val_loss: 0.4329 - val_accuracy: 0.8889
Epoch 4/10
11/11 [==============================] - 279s 27s/step - loss: 0.4418 - accuracy: 0.8935 - val_loss: 0.4020 - val_accuracy: 0.9246
Epoch 5/10
11/11 [==============================] - 279s 27s/step - loss: 0.3934 - accuracy: 0.8994 - val_loss: 0.3612 - val_accuracy: 0.9286
Epoch 6/10
11/11 [==============================] - 237s 22s/step - loss: 0.3607 - accuracy: 0.9083 - val_loss: 0.3473 - val_accuracy: 0.9325
Epoch 7/10
11/11 [==============================] - 279s 27s/step - loss: 0.3264 - accuracy: 0.9290 - val_loss: 0.3172 - val_accuracy: 0.9246
Epoch 

In [11]:
loss, accuracy = model.evaluate(validation_generator)
print('Validation accuracy:', accuracy)

8/8 [==============================] - 102s 13s/step - loss: 0.2736 - accuracy: 0.9405
Validation accuracy: 0.9404761791229248
